In [1]:
%reset -fs
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

import re

from corextopic import corextopic as ct

In [2]:
df = pd.read_csv('eu_tweet_df_eda1.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_17220/3349357611.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_tweet_df_eda1.csv')


In [15]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [4]:
def strip_punc_lower_http(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)

    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower_http('I am, testing #ALEX > than X 10. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html')

'i am testing alex  than x   '

In [5]:
df['text'] = df['text'].apply(lambda x: strip_punc_lower_http(x))

In [6]:
df['text'].head()

0    pounding the streets of redland this morning w...
1    the ukip battle bus is coming please support m...
2    yesterday was th european maritime day today i...
3    thesnp depute leader and candidate for moray a...
4    cacophonic that and fox hunting and dementia t...
Name: text, dtype: object

In [7]:
df_0_500k = df.loc[:500000,['id', 'created_at', 'text']].copy()

In [8]:
df_0_500k

,id,created_at,text
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...
...,...,...,...
499996,959491023046049792,Fri Feb 02 18:17:41 +0000 2018,united states congressman rephuffman took time...
499997,959490706606829568,Fri Feb 02 18:16:25 +0000 2018,were lifting the public sector pay cap mean...
499998,959491686048071680,Fri Feb 02 18:20:19 +0000 2018,a winter scene in the yorkshiredales this morn...
499999,959490755168555008,Fri Feb 02 18:16:37 +0000 2018,russia interfered in the election\nrussia wil...


In [9]:
df_0_500k.to_csv('eu_first500_clean.csv', index = False)

In [10]:
my_stop_words = ['wonderful', 'did', 'didnt', 'issue', 'issues', 'actually', 'id','th',
                'community']

In [11]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

In [12]:
tv_30docs = TfidfVectorizer(stop_words = my_stop_words, min_df=30)
dtm_tv_30docs = tv_30docs.fit_transform(list(df_0_500k['text']))

In [16]:
dtm_tv_30docs.shape

(500001, 16604)

In [17]:
words_30 = list(np.asarray(tv_30docs.get_feature_names_out()))

In [27]:
corex_model = ct.Corex(n_hidden=  50, ### YOUR NUMBER OF TOPICS HERE
                       words=   words_30,   ### YOUR VOCABULARY HERE
                       seed=1
                       )

corex_model.fit(dtm_tv_30docs,       ### YOUR DOCUMENT TERM MATRIX HERE
                words= words_30   , ### YOUR VOCABULARY HERE
                docs= df_0_500k['text']     ### YOUR DOCUMENT SERIES HERE
                )

In [28]:
topics = corex_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: americans, tax, senate, gop, congress, families, republicans, trumpcare, dreamers, bipartisan
1: yellabelly, pilgrim, jamieb, stevemelady, bikerrebel, bostonpoliceuk, bostonboro, inspmorrice, lincsts, maddisongm
2: quintonlabour, harbornelabour, edgbastonclp, birminghammail, amansfield, uhbcomms, birminghampost, swbhnhs, jacqui_smith, daninmich
3: acres, contained, hwy, sonoma, napa, southernlnucomplex, clearlake, sulphurfire, nunsfire, lospadresnf
4: defiaye, tlscadden, yoorwullie, plug, power, primarily, referenced, relating, repeated, rewarded
5: sistercrow, _mattresssale, publicly, protectors, prolonged, prejudice, possibility, pump, plane, opposite
6: baldwin, medal, mick, needing, organized, outoftouch, partial, pens, pillars, pmakela
7: armistice, neglected, negligence, powerful, rec, remembers, semester, shares, spoke, stakes
8: abraham, principle, quietly, rage, receives, refuses, relative, seize, sheriffs, shoe
9: adopting, observed, opponent, outdoor, pages, proposing, pr

In [105]:
corex_model.tc

0.047930982517287404

(500001, 16604)

In [74]:
words_30_semi = list(np.asarray(tv_30docs.get_feature_names_out()))

In [18]:
anchor_words =[
                ['immigration', 'immigrant'],
                ['christian', 'hindu', 'muslim', 'jewish', 'religion'],
                ['trump'],
                ['brexit', 'eu', 'leave'],
                ['women','violence', 'rights'],
                ['health', 'care','insurance', 'mental'],
                ['climate', 'change'],
                ['jobs', 'economy', 'business']
                ]




corex_semi = ct.Corex(n_hidden= 8, ### YOUR NUMBER OF TOPICS HERE
                       words=   words_30,   ### YOUR VOCABULARY HERE
                       seed=1
                       )

corex_semi.fit(dtm_tv_30docs,       ### YOUR DOCUMENT TERM MATRIX HERE
                words= words_30   , ### YOUR VOCABULARY HERE
                docs= df_0_500k['text'],
                anchors=anchor_words,
                anchor_strength = 3
                )

In [19]:
topics_semi = corex_semi.get_topics()
for n,topic in enumerate(topics_semi):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: immigrant, quintonlabour, harbornelabour, edgbastonclp, birminghammail, amansfield, uhbcomms, birminghampost, swbhnhs, jacqui_smith
1: muslim, jewish, christian, religion, yellabelly, pilgrim, jamieb, hindu, stevemelady, bikerrebel
2: daninmich, tuffelhund, sumasteven, functions, interpret, roundup, pas, penalties, perceptions, pioneers
3: eu, brexit, leave, uk, trade, deal, customs, european, union, negotiations
4: violence, victims, prayers, thoughts, vegas, las, attack, women, praying, horrific
5: health, care, mental, insurance, tax, trumpcare, medicaid, americans, gop, premiums
6: climate, change, cop, paris, parisagreement, accord, actonclimate, bonn, deniers, emissions
7: acted, oligarchs, optout, org, overcoming, overreach, overseeing, ozone, paperwork, patent


In [31]:
from corextopic import vis_topic as vt
vt.vis_rep(corex_semi, column_label=words, prefix='topic-model-example')

Print topics in text file


In [22]:
(corex_semi.tc
)

0.06505907678849725